In [1]:
import pandas as pd
import numpy as np

from time import sleep
from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [2]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
raw_df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1HOUR,
                                      str(365) + ' days ago UTC'))

# First 6 column represent Time and OHLCV values
raw_df = raw_df.iloc[:, :6]
raw_df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
raw_df['Time'] = pd.to_datetime(raw_df['Time'], unit='ms')
for column in raw_df.columns:
    if column != 'Time':
        raw_df[column] = raw_df[column].astype(float)

# Set Time as index
raw_df.set_index('Time', inplace=True)
raw_df = raw_df.iloc[:-1]

# Generate Technical Indicators

In [3]:
def get_ta(raw_df):
    df = raw_df.copy()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['SMA200'] = df['Close'].rolling(200).mean()
    df['RSI'] = momentum.rsi(df['Close'])
    df['MACD'] = trend.macd(df['Close'])
    df['MACD_sig'] = trend.macd_signal(df['Close'])
    df['MACD_diff'] = trend.macd_diff(df['Close'])
    df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
    df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
    df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
    df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])
    df.dropna(inplace=True)
    return df


In [4]:
df = get_ta(raw_df)

# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [5]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2023-08-04 20:00:00,29032.05,29081.59,28807.54,28967.51,2246.44732,29190.2252,29261.43585,31.028207,-36.773888,-12.507217,-24.266671,-0.014461,16.903498,40.969397,13.670159
2023-08-04 21:00:00,28967.50,29105.27,28958.18,29056.36,903.22605,29188.3212,29259.48755,40.449313,-40.909348,-18.187643,-22.721705,-0.006928,18.947119,37.118355,13.898464
2023-08-04 22:00:00,29056.37,29094.99,29056.36,29057.62,401.35681,29186.7244,29258.02565,40.573281,-43.582664,-23.266647,-20.316017,-0.008359,20.844767,36.157100,13.538535
2023-08-04 23:00:00,29057.62,29130.35,29056.67,29113.99,484.70925,29185.4130,29257.13555,45.990263,-40.683722,-26.750062,-13.933660,-0.003977,22.099030,34.330915,15.278640
2023-08-05 00:00:00,29114.00,29152.23,29097.17,29118.48,542.47685,29184.0626,29256.35330,46.409295,-37.590663,-28.918182,-8.672480,-0.005168,22.951975,32.989989,16.234012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26 07:00:00,67062.01,67190.00,66879.99,66890.56,1281.20610,65528.8982,66059.62065,66.855861,547.451512,368.773780,178.677732,0.037417,37.202768,10.652208,30.574348
2024-07-26 08:00:00,66890.56,67142.76,66890.56,67106.01,613.50309,65553.0584,66072.93300,69.071530,559.801775,406.979379,152.822396,0.045135,37.997110,10.244567,29.404321
2024-07-26 09:00:00,67106.00,67498.00,67085.77,67296.01,1270.55004,65577.4186,66086.01810,70.917888,578.255096,441.234522,137.020573,0.049965,39.206119,9.598020,32.987196


In [6]:
def get_signal(df):

    #SMA Golden Cross & Death Cross
    sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
    sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
    df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

    # MACD Below 0 Cross
    macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
    macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
    df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

    # RSI Overbought & Oversold
    rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
    rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
    df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

    # ADX Trend Confirmation
    adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
    adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
    df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

    # Momentum Trade
    momentum_buy_cond = (df['Momentum'] > 0) & (df['Momentum'].shift(1) > 0) & (df['Momentum'].shift(2) > 0) & (df['Momentum'].shift(3) > 0) & (df['Momentum'].shift(4) > 0) \
                        & (df['Momentum'].shift(5) > 0) & (df['Momentum'].shift(6) > 0) & (df['Momentum'].shift(7) > 0) & (df['Momentum'].shift(8) > 0) & (df['Momentum'].shift(9) > 0)
    momentum_sell_cond = (df['Momentum'] < 0) & (df['Momentum'].shift(1) < 0) & (df['Momentum'].shift(2) < 0) & (df['Momentum'].shift(3) < 0) & (df['Momentum'].shift(4) < 0) \
                        & (df['Momentum'].shift(5) < 0) & (df['Momentum'].shift(6) < 0) & (df['Momentum'].shift(7) < 0) & (df['Momentum'].shift(8) < 0) & (df['Momentum'].shift(9) < 0)
    df['Signal_momentum'] = np.where(momentum_buy_cond, 1, np.where(momentum_sell_cond, -1, 0))

    df['Buy/Sell Consensus'] = df['Signal_sma'] + df['Signal_macd'] + df['Signal_rsi'] + df['Signal_adx'] + df['Signal_momentum']

    return df

In [7]:
df = get_signal(df)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-04 20:00:00,29032.05,29081.59,28807.54,28967.51,2246.44732,29190.2252,29261.43585,31.028207,-36.773888,-12.507217,...,-0.014461,16.903498,40.969397,13.670159,0,0,0,0,0,0
2023-08-04 21:00:00,28967.50,29105.27,28958.18,29056.36,903.22605,29188.3212,29259.48755,40.449313,-40.909348,-18.187643,...,-0.006928,18.947119,37.118355,13.898464,0,0,0,0,0,0
2023-08-04 22:00:00,29056.37,29094.99,29056.36,29057.62,401.35681,29186.7244,29258.02565,40.573281,-43.582664,-23.266647,...,-0.008359,20.844767,36.157100,13.538535,0,0,0,0,0,0
2023-08-04 23:00:00,29057.62,29130.35,29056.67,29113.99,484.70925,29185.4130,29257.13555,45.990263,-40.683722,-26.750062,...,-0.003977,22.099030,34.330915,15.278640,0,0,0,0,0,0
2023-08-05 00:00:00,29114.00,29152.23,29097.17,29118.48,542.47685,29184.0626,29256.35330,46.409295,-37.590663,-28.918182,...,-0.005168,22.951975,32.989989,16.234012,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26 07:00:00,67062.01,67190.00,66879.99,66890.56,1281.20610,65528.8982,66059.62065,66.855861,547.451512,368.773780,...,0.037417,37.202768,10.652208,30.574348,0,0,-1,1,0,0
2024-07-26 08:00:00,66890.56,67142.76,66890.56,67106.01,613.50309,65553.0584,66072.93300,69.071530,559.801775,406.979379,...,0.045135,37.997110,10.244567,29.404321,0,0,0,1,0,1
2024-07-26 09:00:00,67106.00,67498.00,67085.77,67296.01,1270.55004,65577.4186,66086.01810,70.917888,578.255096,441.234522,...,0.049965,39.206119,9.598020,32.987196,0,0,0,1,1,2


# Run Test Trades on Historcical Data

In [8]:
historical_df = df[df['Buy/Sell Consensus'] != 0]
historical_df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-05 03:00:00,29086.04,29102.45,29082.25,29094.53,430.05398,29177.0162,29255.00915,44.974883,-35.581374,-32.776886,...,-0.005448,25.784243,32.488882,14.889015,0,0,0,-1,0,-1
2023-08-05 04:00:00,29094.52,29096.23,29035.00,29056.01,735.91113,29174.5968,29254.56640,41.641240,-36.882538,-33.598016,...,-0.006131,26.956148,34.760225,14.132106,0,0,0,-1,0,-1
2023-08-05 05:00:00,29056.01,29096.24,29045.28,29096.20,547.06463,29172.8780,29254.11165,46.127959,-34.275614,-33.733536,...,-0.003338,28.044159,33.245412,13.517099,0,0,0,-1,-1,-2
2023-08-05 06:00:00,29096.20,29097.72,29064.22,29066.84,461.01096,29171.4862,29253.43440,43.497053,-34.184655,-33.823760,...,-0.005682,29.024939,32.250437,13.244777,0,0,0,-1,-1,-2
2023-08-05 07:00:00,29066.83,29069.18,29046.22,29052.81,493.94121,29169.5624,29252.58455,42.256762,-34.843025,-34.027613,...,-0.004978,30.088168,33.247780,12.958526,0,0,0,-1,-1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-26 06:00:00,66924.01,67160.00,66874.01,67062.01,697.81043,65507.3672,66045.60780,70.592775,545.661881,324.104347,...,0.023616,36.347322,11.159032,31.568623,0,0,0,1,0,1
2024-07-26 08:00:00,66890.56,67142.76,66890.56,67106.01,613.50309,65553.0584,66072.93300,69.071530,559.801775,406.979379,...,0.045135,37.997110,10.244567,29.404321,0,0,0,1,0,1
2024-07-26 09:00:00,67106.00,67498.00,67085.77,67296.01,1270.55004,65577.4186,66086.01810,70.917888,578.255096,441.234522,...,0.049965,39.206119,9.598020,32.987196,0,0,0,1,1,2


In [9]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0

In [10]:
for idx, row in historical_df.iterrows():
    if (row['Buy/Sell Consensus'] >= 2) & (open_trade == False):
        coin = (balance / row['Close']) *0.999
        bought_price = row['Close']
        open_trade = True
    elif (row['Buy/Sell Consensus'] <= -2) & (open_trade == True):
        open_trade = False
        print(f"PnL:{(row['Close']-bought_price)/bought_price * 100}")
        balance = coin * row['Close'] *0.999
        coin = 0

PnL:-2.064272812910123
PnL:-0.5811474274025049
PnL:-0.43594123937550555
PnL:-0.961584349247738
PnL:-1.8281851075985587
PnL:0.8161880670702112
PnL:3.6446758906559706
PnL:-0.7801320206728518
PnL:26.66736781001343
PnL:-0.37111703639237975
PnL:4.501794165506847
PnL:-2.8569409914083215
PnL:-2.4712501615309947
PnL:-0.06560376239052165
PnL:14.67976366243981
PnL:-4.62065713265652
PnL:-1.4579873160448351
PnL:0.4515956924288818
PnL:-2.029125346527538
PnL:-1.6350823410144273
PnL:-2.805338941471094
PnL:2.5357173872174137
PnL:-2.7780660545973204
PnL:-1.3106624322889444
PnL:14.746034840622505
PnL:22.706302838763058
PnL:4.368010289088295
PnL:-6.624048820736145
PnL:5.6128839433511235
PnL:2.2684726920291087
PnL:-5.161179938646545
PnL:-2.7042173902549704
PnL:6.69416482115294
PnL:-3.2121000391632295
PnL:-2.052649122250086
PnL:3.238354576124288
PnL:-2.090006784881639
PnL:0.8087015743747492
PnL:-2.983511117454115
PnL:-1.4237920886208315
PnL:-1.6141278972252853
PnL:-0.32163742690058483
PnL:-3.86056130148591

In [11]:
balance

1501.8301514026277

In [12]:
1 - 1/1000

0.999

# Run Test Trades on Live Data

In [13]:
raw_df

,Open,High,Low,Close,Volume
Time,,,,,
2023-07-27 13:00:00,29496.42,29502.50,29431.19,29446.02,1066.31824
2023-07-27 14:00:00,29446.01,29446.02,29322.49,29350.00,1714.18262
2023-07-27 15:00:00,29350.00,29361.68,29236.78,29292.01,1816.42610
2023-07-27 16:00:00,29292.00,29333.22,29261.25,29274.93,1012.68895
2023-07-27 17:00:00,29274.93,29292.06,29180.00,29220.00,1363.48368
...,...,...,...,...,...
2024-07-26 07:00:00,67062.01,67190.00,66879.99,66890.56,1281.20610
2024-07-26 08:00:00,66890.56,67142.76,66890.56,67106.01,613.50309
2024-07-26 09:00:00,67106.00,67498.00,67085.77,67296.01,1270.55004


In [14]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0
balance_df = {f"{raw_df.index[-1]}": balance}

In [15]:
while True:
        # Get new hour data
        hour_data = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                                Client.KLINE_INTERVAL_1HOUR,
                                        str(1) + ' hour ago UTC'))

        # First 6 column represent Time and OHLCV values
        hour_data = hour_data.iloc[:, :6]
        hour_data.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

        # Convert first column into datetime
        hour_data['Time'] = pd.to_datetime(hour_data['Time'], unit='ms')
        for column in hour_data.columns:
                if column != 'Time':
                        hour_data[column] = hour_data[column].astype(float)

        # Set Time as index
        hour_data.set_index('Time', inplace=True)

        # Join with old data
        raw_df = pd.concat([raw_df, hour_data])

        # Get TAs and Trading Signals
        df = get_ta(raw_df)
        df = get_signal(df)

        # Simulate Trade
        if (df.iloc[-2, -1] >= 2) & (open_trade == False):
                coin = (balance / df.iloc[-1,0]) *0.999
                balance = 0
                bought_price = df.iloc[-1,0]
                open_trade = True
        elif (df.iloc[-2, -1] <= -2) & (open_trade == True):
                open_trade = False
                print(f"PnL:{(df.iloc[-1,0]-bought_price)/bought_price * 100}")
                balance = coin * df.iloc[-1,0] *0.999
                coin = 0

        balance_df[f'{df.index[-1]}'] =  balance
        print(f"Current Balance: {balance} USD and {coin} BTC at {df.index[-1]}")

        sleep(3600)
     

Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 12:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 13:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 14:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 15:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 16:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 17:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 18:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 19:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 20:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 21:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 22:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-26 23:00:00
Current Balance: 0 USD and 0.014839133543140953 BTC at 2024-07-27 00:00:00
Current Balance: 0 USD an